#### Latent Dirichlet Allocation (LDA)를 이용한 뉴스 데이터 클러스터링
gensim 패키지를 이용한다.--요약지원하는 textrank알고리즘 내장  
--document frequency (DF) 정보가 필요하기 때문에 corpus 와 idx to word 정보가 포함된 dictionary 를 입력해야함. 

In [1]:
import numpy as np
import re
import pickle
from nltk.corpus import stopwords
#from sklearn.datasets import fetch_20newsgroups

In [2]:
#newsData = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))

In [3]:
#with open('./data/news.data', 'wb') as f:
#    pickle.dump(newsData , f, pickle.HIGHEST_PROTOCOL)

In [4]:
with open('./data/news.data', 'rb') as f:
    newsData  = pickle.load(f)

In [5]:
news = newsData.data
print(len(news))
print(news[0])

11314
Well i'm not sure about the story nad it did seem biased. What
I disagree with is your statement that the U.S. Media is out to
ruin Israels reputation. That is rediculous. The U.S. media is
the most pro-israeli media in the world. Having lived in Europe
I realize that incidences such as the one described in the
letter have occured. The U.S. media as a whole seem to try to
ignore them. The U.S. is subsidizing Israels existance and the
Europeans are not (at least not to the same degree). So I think
that might be a reason they report more clearly on the
atrocities.
	What is a shame is that in Austria, daily reports of
the inhuman acts commited by Israeli soldiers and the blessing
received from the Government makes some of the Holocaust guilt
go away. After all, look how the Jews are treating other races
when they got power. It is unfortunate.



In [6]:
print(newsData.target_names)
print(len(newsData.target_names))

['alt.atheism', 'comp.graphics', 'comp.os.ms-windows.misc', 'comp.sys.ibm.pc.hardware', 'comp.sys.mac.hardware', 'comp.windows.x', 'misc.forsale', 'rec.autos', 'rec.motorcycles', 'rec.sport.baseball', 'rec.sport.hockey', 'sci.crypt', 'sci.electronics', 'sci.med', 'sci.space', 'soc.religion.christian', 'talk.politics.guns', 'talk.politics.mideast', 'talk.politics.misc', 'talk.religion.misc']
20


#### preprocessing
영문자가 아닌 문자를 제거한다.

In [7]:
news1 = []
for doc in news:
    news1.append(re.sub("[^a-zA-Z]", " ", doc))

불용어를 제거하고, 모든 단어를 소문자로 변환하고, 길이가 3 이하인 단어를 제거한다.

In [8]:
stop_words = stopwords.words('english')
news2 = []
for doc in news1:
    doc1 = []
    for w in doc.split():
        w = w.lower()
        if len(w) > 3 and w not in stop_words:
            doc1.append(w)
    news2.append(doc1)

In [9]:
print(news2[0])

['well', 'sure', 'story', 'seem', 'biased', 'disagree', 'statement', 'media', 'ruin', 'israels', 'reputation', 'rediculous', 'media', 'israeli', 'media', 'world', 'lived', 'europe', 'realize', 'incidences', 'described', 'letter', 'occured', 'media', 'whole', 'seem', 'ignore', 'subsidizing', 'israels', 'existance', 'europeans', 'least', 'degree', 'think', 'might', 'reason', 'report', 'clearly', 'atrocities', 'shame', 'austria', 'daily', 'reports', 'inhuman', 'acts', 'commited', 'israeli', 'soldiers', 'blessing', 'received', 'government', 'makes', 'holocaust', 'guilt', 'away', 'look', 'jews', 'treating', 'races', 'power', 'unfortunate']


#### doc2bow 생성

In [10]:
from gensim import corpora
from gensim.models.ldamodel import LdaModel as LDA

In [11]:
vocab = corpora.Dictionary(news2) #news data diction화,

In [12]:
#print(dict(vocab))
dict(list(vocab.items())[:10])

{0: 'acts',
 1: 'atrocities',
 2: 'austria',
 3: 'away',
 4: 'biased',
 5: 'blessing',
 6: 'clearly',
 7: 'commited',
 8: 'daily',
 9: 'degree'}

In [13]:
len(vocab)

64281

In [14]:
news_bow = [vocab.doc2bow(s) for s in news2]#document---> bag of word

In [15]:
print(news_bow[0])
#(0,1 ) => voca에서 0번 단어가 1번 쓰였음( acts )

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 2), (22, 2), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 4), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)]


#### LDA 학습

In [16]:
model = LDA(news_bow, num_topics = len(newsData.target_names), id2word=vocab)
#           data                      20개                       

#### Topic별로 문서 분류
doc_topic 행렬에서 가장 큰 colume을 선택한다. Colume에 topic의 score가 부여돼 있다.

In [17]:
doc_topic = model.get_document_topics(news_bow) #문서 대 주제 행렬

In [18]:
dp = np.array(doc_topic[0])
print(dp)
print(dp[np.argmax(dp[:, 1]), 0]) #문서 0은 topic이 7일 확률이 44%다 이런식으로 출력

[[ 7.          0.44350123]
 [ 8.          0.08121175]
 [15.          0.3186287 ]
 [16.          0.14352335]]
7.0


In [19]:
np.sum(dp[:,1]) #위에 문서 0의 %를 모두(일정 확률 이상의) 더해보면 98% 거의 1이 나옴

0.9868650287389755

여기까진 문서 0번만 본거고 for문으로 전체로 봐보자

In [20]:
for i in range(20):
    dp = np.array(doc_topic[i])
    most_likely_topic = int(dp[np.argmax(dp[:, 1]), 0])
    print('문서-{:d} : topic = {:d}'.format(i+1, most_likely_topic+1))

문서-1 : topic = 8
문서-2 : topic = 16
문서-3 : topic = 8
문서-4 : topic = 13
문서-5 : topic = 14
문서-6 : topic = 15
문서-7 : topic = 18
문서-8 : topic = 11
문서-9 : topic = 20
문서-10 : topic = 18
문서-11 : topic = 1
문서-12 : topic = 2
문서-13 : topic = 4
문서-14 : topic = 13
문서-15 : topic = 1
문서-16 : topic = 1
문서-17 : topic = 5
문서-18 : topic = 8
문서-19 : topic = 2
문서-20 : topic = 12


#### topic_term 행렬에서 topic 별로 중요 단어를 표시한다

In [21]:
topic_term = model.get_topic_terms(0, topn=10)
#                      첫번째 주제에서 / 상위 퍼센트 뽑아와
topic_term

[(282, 0.012816965),
 (1657, 0.010865842),
 (627, 0.0097775245),
 (2171, 0.008808282),
 (1870, 0.008058374),
 (1013, 0.007561394),
 (962, 0.007505523),
 (213, 0.0068516405),
 (2123, 0.0067374855),
 (1186, 0.005642815)]

In [22]:
for i in range(len(newsData.target_names)):
    topic_term = model.get_topic_terms(i, topn=10)
    idx = [idx for idx, score in topic_term]
    print('토픽-{:2d} : '.format(i+1), end='')
    for n in idx:
        print('{:s} '.format(vocab[n]), end='')
    print()

토픽- 1 : file program available windows software files entry also version info 
토픽- 2 : drive information also output mail used using please ground wire 
토픽- 3 : encryption security keys public chip government clipper health number information 
토픽- 4 : bike like would good gordon soon pitt ride riding bikes 
토픽- 5 : would like guns right time people year think well could 
토픽- 6 : would government constitution water right congress amendment arms united people 
토픽- 7 : team play hockey league players player teams last myers period 
토픽- 8 : israel jews would israeli arab people right like palestinian state 
토픽- 9 : people think would know president going like something said really 
토픽-10 : freeware expos acid intake xsun points burt tranny trends would 
토픽-11 : armenian armenians people like turkish said would know turkey could 
토픽-12 : jesus bible christian christ faith christians believe would word father 
토픽-13 : would space people science also think time many research world 
토픽-14 : ga